### References :
- https://www.linkedin.com/posts/clotilde-djuikem-8186901a0_maths-et-netflix-ugcPost-7283008969764192257-in4f?utm_source=share&utm_medium=member_desktop
- https://www.kaggle.com/code/mukandkrishna/collaborative-filtering

- https://medium.com/@xaradxarma/book-recommendation-system-8cdb77585b65

- https://www.kaggle.com/code/sagarvarandekar/book-recommendation-collaborative-filtering

- https://www.kaggle.com/code/ahb1104/neural-collaborative-filtering (just principe but neural network in this code is built by pytorch)

- https://medium.com/@sohelrana.aiubPro/enhancing-book-recommendations-leveraging-deep-learning-algorithms-for-personalized-suggestions-a6f209849d36

## Collaborative Filtering

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
books=pd.read_csv('Books.csv')
ratings=pd.read_csv('Ratings.csv')
users=pd.read_csv('Users.csv')

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [ ]:
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
## shape
print("books : {}".format(books.shape))
print("ratings : {}".format(ratings.shape))
print("users : {}".format(users.shape))

books : (271360, 8)
ratings : (1149780, 3)
users : (278858, 3)


##### chkecking Nan

In [ ]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


In [ ]:
users.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


--> We can drop Age because there are many nan values

In [ ]:
users=users.drop(columns='Age')

In [ ]:
ratings.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [ ]:
books=books.drop(columns=['Image-URL-S','Image-URL-M','Image-URL-L'])

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
ratings_with_books = ratings.merge(books,on = 'ISBN')

In [ ]:
ratings_with_books.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276727,0446520802,0.0,The Notebook,Nicholas Sparks,1996,Warner Books
2,276744,038550120X,7.0,A Painted House,JOHN GRISHAM,2001,Doubleday
3,276746,0425115801,0.0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group
4,276746,0449006522,0.0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books


In [ ]:
ratings_with_books.shape

(1031136, 7)

In [ ]:
ratings_with_books.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2


In [ ]:
## counting the number of rating
number_book_rating=ratings_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()
number_book_rating.head()

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [ ]:
## average of book
average_book_rating=ratings_with_books.groupby('Book-Title')['Book-Rating'].mean().reset_index()
average_book_rating.head()

,Book-Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [ ]:
## merge average_book_rating and number_book_rating
data_rating=average_book_rating.merge(number_book_rating,on='Book-Title')
data_rating.head()

,Book-Title,Book-Rating_x,Book-Rating_y
0,A Light in the Storm: The Civil War Diary of ...,2.25,4
1,Always Have Popsicles,0.00,1
2,Apple Magic (The Collector's series),0.00,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00,1
4,Beyond IBM: Leadership Marketing and Finance ...,0.00,1


In [ ]:
data_rating.describe()

,Book-Rating_x,Book-Rating_y
count,241071.000000,241071.000000
mean,2.890137,4.277312
std,3.268984,16.738685
min,0.000000,1.000000
25%,0.000000,1.000000
50%,1.971429,1.000000
75%,5.000000,3.000000
max,10.000000,2502.000000


In [ ]:
#rename column
data_rating=data_rating.rename(columns={'Book-Rating_x':'avg_rating','Book-Rating_y':'num_rating'})

In [ ]:
## filter data t
data_rating=data_rating[data_rating['num_rating']>=50]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
data_rating['avg_rating']=scale.fit_transform(data_rating[['avg_rating']])
data_rating['num_rating']=scale.fit_transform(data_rating[['num_rating']])

In [ ]:
data_rating.head()

,Book-Title,avg_rating,num_rating
203,10 Lb. Penalty,0.216432,0.004486
697,16 Lighthouse Road,0.160987,0.006117
764,1984,0.510662,0.095432
818,1st to Die: A Novel,0.390020,0.187194
946,2010: Odyssey Two,0.265205,0.016313


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
## features matrix
features=data_rating[['avg_rating','num_rating']]
similarities=cosine_similarity(features)
similarities

array([[1.        , 0.99985111, 0.98657825, ..., 0.96632113, 0.99989269,
        0.99980296],
       [0.99985111, 1.        , 0.98924905, ..., 0.97061786, 0.99949101,
        0.99931156],
       [0.98657825, 0.98924905, 1.        , ..., 0.99537212, 0.98408022,
        0.98314251],
       ...,
       [0.96632113, 0.97061786, 0.99537212, ..., 1.        , 0.96244746,
        0.96102246],
       [0.99989269, 0.99949101, 0.98408022, ..., 0.96244746, 1.        ,
        0.99998647],
       [0.99980296, 0.99931156, 0.98314251, ..., 0.96102246, 0.99998647,
        1.        ]])

In [ ]:
def recommend_books(book_title, top_n=5):
    if book_title not in data_rating['Book-Title'].values:
        return f"Book '{book_title}' not found in the dataset."

    # Find the index of the book
    book_idx = data_rating[data_rating['Book-Title'] == book_title].index[0]

    # Get similarity scores for the book
    similarity_scores = list(enumerate(similarities[book_idx]))

    # Sort by similarity scores in descending order
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top_n most similar books (excluding the book itself)
    # Use .iloc to access rows by position based on sorted similarity scores
    top_books = [data_rating.iloc[i[0]]['Book-Title'] for i in sorted_scores[1:top_n+1]]

    return top_books , sorted_scores[1:top_n+1]

In [ ]:
## test with book titles
print(recommend_books('16 Lighthouse Road'))

(['Harry Potter and the Goblet of Fire (Book 4)', 'Night Sins', "Sophie's World: A Novel About the History of Philosophy", 'Girl in Hyacinth Blue', 'D Is for Deadbeat (Kinsey Millhone Mysteries (Paperback))'], [(789, 0.9999999958204071), (1195, 0.9999999913353221), (1564, 0.9999999837110765), (741, 0.999999977016827), (453, 0.9999999675222963)])


In [ ]:
from sklearn.metrics import mean_squared_error

# Function to calculate MSE
def calculate_mse(book_title, top_n=5):
    if book_title not in data_rating['Book-Title'].values:
        return f"Book '{book_title}' not found in the dataset."


    book_idx = data_rating[data_rating['Book-Title'] == book_title].index[0]

    similarity_scores = list(enumerate(similarities[book_idx]))

    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]

    actual_ratings = data_rating.iloc[top_indices]['avg_rating'].values
    predicted_ratings = [similarities[book_idx][i] for i in top_indices]

    mse = mean_squared_error(actual_ratings, predicted_ratings)

    return mse


In [ ]:
mse_value = calculate_mse('1984', top_n=5)
print(f"Mean Squared Error: {mse_value}")

Mean Squared Error: 0.4881009842147631


## USING NEURAL NETWORK

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dropout, Dense, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


In [ ]:
ratings_with_books.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276727,0446520802,0.0,The Notebook,Nicholas Sparks,1996,Warner Books
2,276744,038550120X,7.0,A Painted House,JOHN GRISHAM,2001,Doubleday
3,276746,0425115801,0.0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group
4,276746,0449006522,0.0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books


In [ ]:
## Year-Of-Publication
ratings_with_books['Year-Of-Publication'].unique()

array([2002, 1996, 2001, 1994, 1995, 2003, 1997, 1998, 1999, 1992,    0,
       1976, 1991, 1983, 1990, 1988, 1993, 2000, 1989, 1987, 1986, 1981,
       2004, 1984, 1982, 1974, 1985, 1972, 1979, 1957, 1958, 1960, 1963,
       1969, 1978, 1977, 1980, 1959, 1975, 1973, 1965, 1971, 1955, 1970,
       1953, 1966, 1968, 1930, 1940, 1964, 1952, 1961, 1946, 1941, 1962,
       1954, 1920, 1967, 1956, 1942, 1951, 1948, 1950, 2005, 1943, 1937,
       1923, 1945, 1947, 1936, 2030, 2011, 1925, 2020, 1939, 1926, 1938,
       1911, 1904, 1949, 1932, 1929, 1928])

In [ ]:
# Convert 'Year-Of-Publication' to numeric, handling errors
ratings_with_books['Year-Of-Publication'] = pd.to_numeric(ratings_with_books['Year-Of-Publication'], errors='coerce')

# Filter out rows with invalid 'Year-Of-Publication' (now NaN)
ratings_with_books = ratings_with_books.dropna(subset=['Year-Of-Publication'])

ratings_with_books = ratings_with_books[
    (ratings_with_books['Year-Of-Publication'] < 2024) &
    (ratings_with_books['Year-Of-Publication'] != 0)
]

In [ ]:
user_ids=ratings_with_books['User-ID'].unique()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
book_ids=ratings_with_books['ISBN'].unique()
book2book_encoded = {x: i for i, x in enumerate(book_ids)}
book_encoded2book = {i: x for i, x in enumerate(book_ids)}

In [ ]:
print(user2user_encoded)
print("//////////////////////")
print(userencoded2user)
print("//////////////////////")
print(book2book_encoded)
print("//////////////////////")
print(book_encoded2book)

In [ ]:
ratings_with_books['user'] = ratings_with_books['User-ID'].map(user2user_encoded)
ratings_with_books['book'] = ratings_with_books['ISBN'].map(book2book_encoded)

In [ ]:
## features and targets
X=ratings_with_books[['user','book']]
y=ratings_with_books['Book-Rating']
print(X.shape)
print(y.shape)

(486217, 2)
(486217,)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Parameters
embedding_size=10
n_users = len(user2user_encoded)
n_books = len(book2book_encoded)

# User embedding
user_input = Input(shape=(1,))
user_embedding = Embedding(n_users, embedding_size,embeddings_regularizer=l2(1e-6))(user_input)
user_vec = Flatten()(user_embedding)

# Book embedding
book_input = Input(shape=(1,))
book_embedding = Embedding(n_books, embedding_size,embeddings_regularizer=l2(1e-6))(book_input)
book_vec = Flatten()(book_embedding)

# Concatenate user and book embeddings
concat = Concatenate()([user_vec, book_vec])

# Neural network layers for predictions
dense_1 = Dense(64, activation='relu')(concat)
dense_2 = Dense(32, activation='relu')(dense_1)

# Output layer
output = Dense(1, activation='sigmoid')(dense_2)

model = Model(inputs=[user_input, book_input], outputs=output)
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0005), metrics=['mae'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 10)          │        569,040 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 10)          │        567,940 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 10)             │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 10)             │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 20)             │              0 │ flatten[0][0],         │
│                           │                        │                │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          1,344 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │             33 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,140,437 (4.35 MB)

 Trainable params: 1,140,437 (4.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train model
history = model.fit(
    [X_train['user'], X_train['book']],
    y_train/10,
    epochs=5,
    batch_size=32,
    validation_data=([X_test['user'], X_test['book']], y_test/10),
    verbose=1
)

Epoch 1/5
12156/12156 ━━━━━━━━━━━━━━━━━━━━ 299s 24ms/step - loss: 0.1306 - mae: 0.3125 - val_loss: 0.1204 - val_mae: 0.2832
Epoch 2/5
12156/12156 ━━━━━━━━━━━━━━━━━━━━ 330s 25ms/step - loss: 0.1081 - mae: 0.2606 - val_loss: 0.1208 - val_mae: 0.2847
Epoch 3/5
12156/12156 ━━━━━━━━━━━━━━━━━━━━ 308s 24ms/step - loss: 0.1022 - mae: 0.2478 - val_loss: 0.1226 - val_mae: 0.2762
Epoch 4/5
12156/12156 ━━━━━━━━━━━━━━━━━━━━ 331s 25ms/step - loss: 0.0950 - mae: 0.2323 - val_loss: 0.1262 - val_mae: 0.2789
Epoch 5/5
12156/12156 ━━━━━━━━━━━━━━━━━━━━ 309s 24ms/step - loss: 0.0877 - mae: 0.2174 - val_loss: 0.1295 - val_mae: 0.2739


In [ ]:
model.evaluate([X_test['user'], X_test['book']], y_test)

3039/3039 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.1291 - mae: 0.2735


[0.12950535118579865, 0.273885041475296]

### problem : overfitting

solution  : dropout, batchnormalization , early stopping , increase embeddings size (I don't have more enough time to train :D )